<a href="https://colab.research.google.com/github/nimrashaheen001/Programming_for_AI/blob/main/Lab_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Traditional Machine Learning Implementation**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# Load the dataset
titanic_data = pd.read_csv('titanic.csv')

#print(titanic_data.isnull().sum())
# Handle missing values=
titanic_data['Age'].fillna(titanic_data['Age'].mean(), inplace=True)
titanic_data['Embarked'].fillna(titanic_data['Embarked'].mode()[0], inplace=True)

#sex = pd.get_dummies(train['Sex'],dtype=int)
titanic_data = pd.get_dummies(titanic_data, columns=['Sex', 'Embarked'], dtype=int)

# Select features and target variable
X = titanic_data[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S']]
y = titanic_data[['Survived']]


# Step 4: Split the Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Train the KNN Model
knn = KNeighborsClassifier(n_neighbors=5)  # You can tune n_neighbors
knn.fit(X_train, y_train)

# Step 7: Make Predictions
y_pred = knn.predict(X_test)
#print("Predicted Values:")
#print(y_pred )

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.4f}")

# Step 8: Evaluate the Model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))




Model Accuracy: 0.7095
Confusion Matrix:
[[87 18]
 [34 40]]


<ipython-input-2-35ad67740960>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  titanic_data['Age'].fillna(titanic_data['Age'].mean(), inplace=True)
<ipython-input-2-35ad67740960>:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inpl

# **Sklearn Pipeline Implementation**

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# Load the dataset
data = pd.read_csv('titanic.csv')

#print("Original DataFrame:")
#print(data)
#print(data.shape)


def impute_embarked(X):
    X['Embarked'] = X['Embarked'].fillna(X['Embarked'].mode()[0])  # Fill missing values
    return X

# Custom function to create the FamilySize feature
def create_family_size(X):
    X['FamilySize'] = X['SibSp'] + X['Parch'] + 1  # Adding 1 for the individual themselves
    return X

# Custom function to drop specified columns
def drop_columns(X):
    return X.drop(['SibSp', 'Parch'], axis=1)

# Function to create FamilySize and drop SibSp and Parch columns
def family_size(X):
    X = create_family_size(X)
    X = drop_columns(X)
    return X

# Create pipelines for Age
age_pipeline = Pipeline(steps=[
    ('age_imputer', SimpleImputer(strategy='mean')),  # Impute Age
    ('age_scaler', MinMaxScaler())  # Scale Age
])

fare_pipeline = Pipeline(steps=[
    #('fare_imputer', SimpleImputer(strategy='mean')),  # Impute Fare
    ('fare_scaler', MinMaxScaler())  # Scale Fare
])

family_size_pipeline = Pipeline(steps=[
    ('family_size_creator', FunctionTransformer(family_size)),
    ('family_size_scaler', MinMaxScaler()),  # Scale Family_Size
])

embarked_pipeline = Pipeline(steps=[
    ('embarked_imputer', FunctionTransformer(impute_embarked)),  # Impute Embarked
    ('embarked_onehot', OneHotEncoder())  # One-hot encode Embarked
])

# Create a ColumnTransformer to preprocess the data
preprocessor = ColumnTransformer(transformers=[
    ('drop', 'drop', ['PassengerId', 'Name', 'Ticket', 'Cabin']),
    ('age_encoder', age_pipeline, ['Age']),
    ('fare_encoder', fare_pipeline, ['Fare']),
    ('family_size', family_size_pipeline, ['SibSp', 'Parch']),  # Process FamilySize
    ('embarked_encoder', embarked_pipeline, ['Embarked']),
    ('sex_encoder', OneHotEncoder(), ['Sex']),
    ('scaler', MinMaxScaler(), ['Pclass']),  # Scale Pclass
], remainder='passthrough')

# Create a complete pipeline that includes preprocessing and the classifier
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Preprocessing steps
    ('classifier', KNeighborsClassifier(n_neighbors=5))  # KNN Classifier
])

X = data.drop('Survived', axis=1)
y = data['Survived']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.2f}")

# Step 8: Evaluate the Model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))



Model Accuracy: 0.80
Confusion Matrix:
[[90 15]
 [21 53]]


# **Random UnderSampling**

Why Use Random Undersampling?

Imbalanced datasets can cause machine learning models to favor the majority class.
Undersampling helps balance the dataset by reducing the number of samples in the majority class.
Trade-off: Risk of losing important data (since some majority-class samples are removed).

In [ ]:

from collections import Counter # Used to count the occurrences of each class in the dataset, providing a quick summary of class distribution.

from sklearn.datasets import make_classification#A utility function to generate synthetic datasets for classification tasks.

from imblearn.under_sampling import RandomUnderSampler #A technique from the imbalanced-learn library to balance datasets by undersampling the majority class.

X, y = make_classification(n_classes=2, class_sep=2, weights=[0.1, 0.9], n_informative=3, n_redundant=1, flip_y=0,n_features=20, n_clusters_per_class=1, n_samples=1000, random_state=10)

#n_classes=2: The dataset will have 2 classes (binary classification).
#class_sep=2: Controls the separation between the two classes. A larger value makes them more separable.
#weights=[0.1, 0.9]: Specifies the class distribution:
#10% of samples will belong to class 0.
#90% of samples will belong to class 1 (imbalanced dataset).
#n_informative=3: Number of informative features (important for classification).
#_redundant=1: Number of redundant features (linear combinations of informative features).
#flip_y=0: No noise is added to the labels (i.e., no label flipping).
#n_features=20: Total number of features in the dataset.
#n_clusters_per_class=1: Each class will have a single cluster of points.
#n_samples=1000: The dataset will contain 1000 samples in total.
#random_state=10: Ensures reproducibility of the generated dataset.

print('Original dataset shape %s' % Counter(y))

#Counter(y): Counts the occurrences of each class in y.
#Prints the original distribution of classes. Since weights=[0.1, 0.9], we expect approximately:
#100 samples of class 0.
#900 samples of class 1.

rus = RandomUnderSampler(random_state=42) #Creates an instance of RandomUnderSampler
X_res, y_res = rus.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_res))

Original dataset shape Counter({1: 900, 0: 100})
Resampled dataset shape Counter({0: 100, 1: 100})


# **Random OverSampling**

1. What is RandomOverSampler?

RandomOverSampler is a method from the imbalanced-learn library.
It balances imbalanced datasets by increasing the number of samples in the minority class.
How it works:
It randomly duplicates samples from the minority class until both classes have the same number of samples.

Advantages of Oversampling
Balances the Dataset:

Helps prevent models from being biased toward the majority class.
Retains Minority Class Information:
Unlike undersampling, no minority-class samples are removed.

Trade-Offs
Risk of Overfitting:

Since oversampling duplicates existing samples, the model might overfit to the minority class.
Increased Dataset Size:
The number of samples grows, potentially increasing memory and computation requirements.

In [ ]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
X, y = make_classification(n_samples=100, n_features=2, n_informative=2, n_redundant=0, n_repeated=0, n_classes=3, n_clusters_per_class=1, weights=[0.01, 0.05, 0.94], class_sep=0.8, random_state=0)
print('Original dataset shape %s' % Counter(y))
#print(X)
#print(y)


ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)


print('Resampled dataset shape %s' % Counter(y_resampled))
#print(X_resampled)
#print(y_resampled)
#print(sorted(Counter(y_resampled).items()))

Original dataset shape Counter({2: 94, 1: 5, 0: 1})
Resampled dataset shape Counter({2: 94, 1: 94, 0: 94})


# **SMOTE OverSampling**


What is SMOTE?

SMOTE (Synthetic Minority Oversampling Technique):
Generates synthetic samples for minority classes by:

Selecting a sample from the minority class.

Finding its k-nearest neighbors.

Creating a synthetic sample along the line joining the original sample and one of its neighbors.

In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
X, y = make_classification(n_samples=5000, n_features=2, n_informative=2, n_redundant=0, n_repeated=0, n_classes=3, n_clusters_per_class=1, weights=[0.01, 0.05, 0.94], class_sep=0.8, random_state=0)
print('Original dataset shape %s' % Counter(y))


X_resampled, y_resampled = SMOTE().fit_resample(X, y)
print('SMOTE Resampled dataset shape %s' % Counter(y_resampled))


#print(sorted(Counter(y_resampled).items()))

#X_resampled_adasyn, y_resampled_adasyn = ADASYN().fit_resample(X, y)
#print('ADASYN Resampled dataset shape %s' % Counter(y_resampled_adasyn))
#print(sorted(Counter(y_resampled).items()))


Original dataset shape Counter({2: 4674, 1: 262, 0: 64})
SMOTE Resampled dataset shape Counter({2: 4674, 1: 4674, 0: 4674})


# Lab Tasks

Perform following operations on the Titanic Dataset:

*   Apply Random Undersampling to balance the dataset. (https://imbalanced-learn.org/stable/references/generated/imblearn.under_sampling.RandomUnderSampler.html)
*   Apply Random Oversampling and SMOTE (Synthetic Minority Oversampling Technique) to balance the dataset. (https://imbalanced-learn.org/stable/over_sampling.html)
*   Remove Outliers from the dataset.

*   Apply k-fold cross validation on the dataset and display fold wise results.

Perform the steps mentioned above on the following dataset

https://www.kaggle.com/datasets/kamilpytlak/personal-key-indicators-of-heart-disease

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import zscore
from collections import Counter

# Step 1: Generate a synthetic dataset
X, y = make_classification(
    n_samples=1000,
    n_features=10,
    n_informative=5,
    n_redundant=2,
    n_classes=3,
    n_clusters_per_class=1,
    weights=[0.2, 0.5, 0.3],
    random_state=42
)

# Convert to a DataFrame for easier manipulation
data = pd.DataFrame(X, columns=[f'Feature_{i}' for i in range(X.shape[1])])
data['Target'] = y

# Step 2: Remove outliers using Z-score method
def remove_outliers(df, threshold=3):
    """Removes rows with z-scores above the threshold."""
    z_scores = np.abs(zscore(df.iloc[:, :-1]))  # Compute z-scores for feature columns
    mask = (z_scores < threshold).all(axis=1)  # Mask rows without extreme z-scores
    return df[mask]

# Remove outliers
data_cleaned = remove_outliers(data)
X_cleaned = data_cleaned.iloc[:, :-1].values
y_cleaned = data_cleaned['Target'].values

print(f"Original dataset shape: {Counter(y)}")
print(f"Cleaned dataset shape: {Counter(y_cleaned)}")

# Step 3: Apply k-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold CV

# Create a model
model = RandomForestClassifier(random_state=42)

# Perform cross-validation
fold_results = []
for fold, (train_index, test_index) in enumerate(kf.split(X_cleaned), 1):
    X_train, X_test = X_cleaned[train_index], X_cleaned[test_index]
    y_train, y_test = y_cleaned[train_index], y_cleaned[test_index]

    # Train and evaluate the model
    model.fit(X_train, y_train)
    accuracy = model.score(X_test, y_test)
    fold_results.append(accuracy)
    print(f"Fold {fold} Accuracy: {accuracy:.2f}")

# Step 4: Display overall results
print(f"\nFold-wise accuracies: {fold_results}")
print(f"Mean accuracy: {np.mean(fold_results):.2f}")
print(f"Standard deviation: {np.std(fold_results):.2f}")


Original dataset shape: Counter({1: 495, 2: 303, 0: 202})
Cleaned dataset shape: Counter({1: 484, 2: 292, 0: 187})
Fold 1 Accuracy: 0.96
Fold 2 Accuracy: 0.97
Fold 3 Accuracy: 0.96
Fold 4 Accuracy: 0.96
Fold 5 Accuracy: 0.95

Fold-wise accuracies: [0.9637305699481865, 0.9740932642487047, 0.9637305699481865, 0.9635416666666666, 0.953125]
Mean accuracy: 0.96
Standard deviation: 0.01
